In [2]:
import math
import numpy as np
import yaml
from astropy.constants import G, M_earth, R_earth
from astropy import units as u
mu = G.value*M_earth.value
import time
import datetime
from astropy import coordinates as coord
from astropy.time import Time

#Loading content from launch state vector file
yaml_file = open("Transporter3_state_vectors.yaml", 'r')
yaml_content = yaml.full_load(yaml_file)

#Variables declaration
sat_id= '1' #To retrieve the required data from txt file i.e. for the first line of TLE which includes catalog number and classification of the satellite
sat = 0 # Changing the value corresponding to this variable will allow the user to generate the Keplerian elements corresponding to the index number mentioned in the yaml file.

#Retrieving data from the file content
r_test =  yaml_content['deployments'][sat]['r_ecef_m']
v_test =  yaml_content['deployments'][sat]['v_ecef_m_per_s']
sat_name  =  yaml_content['deployments'][sat]['name']
date= yaml_content['deployments'][sat]['date']
ballistic_coeff= yaml_content['deployments'][sat]['ballistic_coef_kg_per_m2']
date= yaml_content['deployments'][sat]['date']
s = str(date.date())
a =time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())

#Convert ECEF to ECI system
now = Time(date)
pos = r_test*u.m
vel = v_test*u.m/u.s
gcrs = coord.ITRS(x=pos[0], y=pos[1], z=pos[2], v_x=vel[0], v_y=vel[1], v_z=vel[2], representation_type='cartesian', differential_type='cartesian', obstime=now)
itrs = gcrs.transform_to(coord.GCRS(obstime=now))

r_test = (itrs.cartesian.xyz)

v_test = (itrs.cartesian.differentials)
v1 = v_test['s'].d_x
v2 = v_test['s'].d_y
v3 = v_test['s'].d_z
v1= (v1.value)*1000
v2= (v2.value)*1000
v3= (v3.value)*1000

#Declaring the new set of coordinates in the respective variables
v_test = [v1,v2,v3]
r_test = r_test.value 


#Calculate r_test^2 and v_test^2
def square(r_test):
    return [i ** 2 for i in r_test] 
var= square(r_test)
var_res= var[0]+var[1]+var[2]
r_test_resultant= math.sqrt(var_res)

def square(v_test):
    return [j ** 2 for j in v_test] 
var_v= square(v_test)
var_res_v= var_v[0]+var_v[1]+var_v[2]
v_test_resultant= math.sqrt(var_res_v)


# Calculate radial velocity
v_radial = (np.dot(r_test, v_test))/r_test_resultant 

# If radial velocity is positive the satellite is flying away from perigee, and if negative then flying towards the perigee
 
#Calculate angular momentum
 
h= np.cross(r_test, v_test)
h_res= h[0]*h[0] + h[1]*h[1] + h[2]*h[2]
h_resultant= math.sqrt(h_res)

# Calculate inclination
inc_rad= math.acos(h[2]/h_resultant)
inc_deg= math.degrees(inc_rad)
inc_deg= '{:.4f}'.format(inc_deg)


# Calculate N-vector
k=[0,0,1]
N= np.cross(k, h)
N_final= N[0]*N[0] + N[1]*N[1] + N[2]* N[2]
N_resultant = math.sqrt(N_final)
 
#Calculate RAAN
RAAN_rad = math.acos(N[0]/N_resultant)
RAAN_1 = math.degrees(RAAN_rad)
RAAN_2 = 360-(math.degrees(RAAN_rad))
if N[1] < 0:
    RAAN_value = RAAN_2 
else:
    RAAN_value = RAAN_1
RAAN_value= '{:.4f}'.format(RAAN_value)

    
#Calculate Eccentricity 
mu_r= mu
ecc_s1= ((var_res_v)- (mu_r/r_test_resultant))
ecc_s2 = np.dot(r_test, ecc_s1)
ecc_s3 = np.dot(r_test, v_test)
ecc_s4 = np.dot(ecc_s3, v_test)
ecc = (ecc_s2 - ecc_s4)/mu_r
ecc_final = ecc[0]**2 + ecc[1]**2 + ecc[2]**2
ecc_resultant = math.sqrt(ecc_final)


#Calculate Argument of Perigee (AOP)
aop_s1 = np.dot(N, ecc)
aop_s2 = N_resultant* ecc_resultant
aop_rad = math.acos(aop_s1/aop_s2)
aop_resultant= math.degrees(aop_rad)
aop_1 = aop_resultant
aop_2 = 360- (aop_resultant)
if ecc[2] >= 0:
    aop_value = aop_1 
else:
    aop_value = aop_2


#Calculate True Anomaly (TA)
TA_s1 = np.dot(ecc, r_test)
TA_s2 = ecc_resultant * r_test_resultant
TA_radian = math.acos(TA_s1/TA_s2)
TA_resultant = math.degrees(TA_radian)
TA_1 = TA_resultant
TA_2 = 360- TA_resultant
if v_radial > 0:
    TA_value = TA_1 
else:
    TA_value = TA_2


#Calculate perigee and apogee radii
peri_radii = (h_res)/(mu_r * (1+ecc_resultant))
apo_radii  = (h_res)/(mu_r * (1-ecc_resultant))


#Semi-major axis of orbit
semimajor = (peri_radii+apo_radii)/2


#Period of the orbit
orbit_period = (2 * math.pi * (semimajor**1.5))/(mu_r**0.5)
time_period= orbit_period/(3600*24)
mean_motion = 1/time_period
mean_motion = '{:.8f}'.format(mean_motion)


#Calculating B-star
bstar= (0.0784829496969028/ballistic_coeff)
btrial = bstar*10000000
str_b = str(btrial)
bal =str_b.split(".")


#Retrieve information for first line of TLE
d1 = {}
dicm = {}
x = 0
count = len(open('Sat_info.txt').readlines())
with open('Sat_info.txt','r',encoding='utf-8') as f:
    h1 = f.readline().strip().replace(':','')
    if h1 == 'Information':
        for i in range(count-1):
            f1 = f.readline()
            if f1 != '\n':
                f2 = f1.strip().split(": ")
                d1[f2[0]] = f2[1]
            else:
                dicm[f'{x}'] = d1
                d1 = {}
                x+=1
                continue
        dicm[f'{x}'] = d1
catalog= dicm[sat_id]['Catalog no']
classification= dicm[sat_id]['Classification']
launch_no= dicm[sat_id]['Launch number of year']
launch_piece= dicm[sat_id]['Piece of launch']


#Truncate results for TLE
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier
#inc_deg = truncate(inc_deg,4)
#RAAN_value = truncate(RAAN_value,4)
#ecc_resultant= truncate(ecc_resultant,4)
aop_value= truncate(aop_value,4)
#mean_motion= truncate(mean_motion,8)

#Convert it to strings for printing in TLE format
str1 = str(aop_value).zfill(8)
str2 = str(inc_deg).zfill(8)
str3 = str(RAAN_value).zfill(8).rjust(9)
str4 = str(mean_motion).ljust(11)
str5 = str(date).split(" ")
str_catalog = str(catalog)
str_classification = str(classification)
str_lno = str(launch_no)
str_lp = str(launch_piece)
str_bal = bal[0]


line1= str(1)+" "+str_catalog+str_classification+" "+str5[0][2]+str5[0][3]+"000"+str_lp+"  "+" "+str5[0][2]+str5[0][3]+"000.00000000"+" "+" .00000000"+" "+" 00000-0"+"  "+str_bal+"-"+"7"+" "+"0"+" "+" 000"

line2 = str(2)+" "+str_catalog+" "+str2+str3+" "+"{:.7f}".format(ecc_resultant).split(".")[-1]+" "+str1+" "+"000.0000"+" "+str4+"00000"


#Checksum Modulo function

def checksum(line1):
    L = line1.strip()
    cksum = 0
    for i in range(68):
        c = L[i]
        if c == ' ' or c == '.' or c == '+' or c.isalpha():
            continue
        elif c == '-':
            cksum = cksum + 1
        else:
            cksum = cksum + int(c)

    cksum %= 10
    return cksum
c1= (checksum(line1))
str_ck1 = str(c1)

def checksum(line2):
    L = line2.strip()
    cksum = 0
    for i in range(68):
        c = L[i]
        if c == ' ' or c == '.' or c == '+' or c.isalpha():
            continue
        elif c == '-':
            cksum = cksum + 1
        else:
            cksum = cksum + int(c)

    cksum %= 10
    return cksum
c2= (checksum(line2))
str_ck2= str(c2)

#Adding the modulo values to TLE
line1= str(1)+" "+str_catalog+str_classification+" "+str5[0][2]+str5[0][3]+"000"+str_lp+"  "+" "+str5[0][2]+str5[0][3]+"000.00000000"+" "+" .00000000"+" "+" 00000-0"+"  "+str_bal+"-"+"7"+" "+"0"+" "+" 000"+str_ck1

line2 = str(2)+" "+str_catalog+" "+str2+str3+" "+"{:.7f}".format(ecc_resultant).split(".")[-1]+" "+str1+" "+"000.0000"+" "+str4+" 0000"+str_ck2
print(sat_name)
print(line1)
print(line2)
#Creating a txt file of the TLE generated
text = sat_name + "\n" + line1 +"\n" + line2
with open(r'C:\Users\VIVO\Documents\State2TLE\TLE_generated.txt', 'w') as f:
    f.write(text)

UNICORN-2E
1 99977U 22000A   22000.00000000  .00000000  00000-0  18664-7 0  0004
2 99977 096.1710 245.9700 0153654 305.6415 000.0000 15.46968930 00003
